# Athena with nested data types

[AWS Data Wrangler](https://github.com/awslabs/aws-data-wrangler) inherits some deep nested types limitations of [Apache Arrow](https://arrow.apache.org/). A good alternative to overcame it is counting on Athena's help to unnest this complex data types before load it in some Pandas Dataframe.

This tutotial will expose some useful features for this purpose.

### Target Dataset:

```sql
WITH dataset AS (
  SELECT ARRAY[
    CAST(ROW('ARN1', 'ACCOUTID1', 'TYPE1') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN2', 'ACCOUTID2', 'TYPE2') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN3', 'ACCOUTID3', 'TYPE3') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR))
  ] AS your_field
)
SELECT
  *
FROM dataset
```

In [1]:
import awswrangler as wr

### Unnesting the inner struct

In [2]:
sql = """
WITH dataset AS (
  SELECT ARRAY[
    CAST(ROW('ARN1', 'ACCOUTID1', 'TYPE1') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN2', 'ACCOUTID2', 'TYPE2') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN3', 'ACCOUTID3', 'TYPE3') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR))
  ] AS your_field
)
SELECT
  transform(your_field, x -> x.arn) AS arn,
  transform(your_field, x -> x.accountid) AS accountid,
  transform(your_field, x -> x.type) AS type
FROM dataset
"""

df = wr.pandas.read_sql_athena(sql)
df.head()

,arn,accountid,type
0,"[ARN1, ARN2, ARN3]","[ACCOUTID1, ACCOUTID2, ACCOUTID3]","[TYPE1, TYPE2, TYPE3]"


In [3]:
df.iloc[0].arn[0]

'ARN1'

### Unnesting the outer array (Only with CTAS approach)

In [4]:
sql = """
WITH dataset AS (
  SELECT ARRAY[
    CAST(ROW('ARN1', 'ACCOUTID1', 'TYPE1') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN2', 'ACCOUTID2', 'TYPE2') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN3', 'ACCOUTID3', 'TYPE3') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR))
  ] AS your_field
)
SELECT t.your_field
FROM dataset, UNNEST(your_field) as t(your_field)
"""

df = wr.pandas.read_sql_athena(sql, ctas_approach=True)
df.head()

,your_field
0,"{'arn': 'ARN1', 'accountid': 'ACCOUTID1', 'typ..."
1,"{'arn': 'ARN2', 'accountid': 'ACCOUTID2', 'typ..."
2,"{'arn': 'ARN3', 'accountid': 'ACCOUTID3', 'typ..."


In [5]:
df.iloc[0].your_field["arn"]

'ARN1'

### Unnesting the outer array and the inner struct (Fully unnested)

In [6]:
sql = """
WITH dataset AS (
  SELECT ARRAY[
    CAST(ROW('ARN1', 'ACCOUTID1', 'TYPE1') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN2', 'ACCOUTID2', 'TYPE2') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR)),
    CAST(ROW('ARN3', 'ACCOUTID3', 'TYPE3') AS ROW(arn VARCHAR, accountid VARCHAR, type VARCHAR))
  ] AS your_field
)
SELECT
  t.your_field.arn,
  t.your_field.accountid,
  t.your_field.type
FROM dataset, UNNEST(your_field) as t(your_field)
"""

df = wr.pandas.read_sql_athena(sql)
df.head()

,arn,accountid,type
0,ARN1,ACCOUTID1,TYPE1
1,ARN2,ACCOUTID2,TYPE2
2,ARN3,ACCOUTID3,TYPE3


In [7]:
df.iloc[0].arn

'ARN1'